In [ ]:
!apt update && !apt upgrade
!apt install git wget libeigen3-dev -y

In [ ]:
!git clone https://github.com/guochengqian/Magic123.git

In [ ]:
%cd Magic123

In [ ]:
!git pull origin main

In [ ]:
%pip install imageio_ffmpeg

In [ ]:
%pip install -r requirements.txt

In [ ]:
%pip install  torch==2.0.0+cu118 torchvision==0.15.1+cu118 --index-url https://download.pytorch.org/whl/cu118 

In [ ]:
%cd pretrained/zero123
!wget https://huggingface.co/cvlab/zero123-weights/resolve/main/105000.ckpt
%cd ../../

In [ ]:
!mkdir -p pretrained/midas
%cd pretrained/midas
!wget https://github.com/isl-org/MiDaS/releases/download/v3_1/dpt_beit_large_512.pt
%cd ../../

In [ ]:
%env CUDA_VISIBLE_DEVICES=0
workspace ="werewolf_2"
DATA_DIR ='/notebooks/threestudio/load/images'
image_path = f'{DATA_DIR}/rgba.png'
image_path_without_rgba = f'{DATA_DIR}/{workspace}.png'
save_path = f'./output/{workspace}'
prompt = '''
3D render of a cute litte werewolf ,fur,
full body visible, model for painting in underwear
'''
# If the image contains non-front-facing objects, specifying the approximate elevation 
# and azimuth angle by setting data.default_elevation_deg and data.default_azimuth_deg can be helpful. 
# In threestudio, top is elevation +90 and bottom is elevation -90; left is azimuth -90 and right is azimuth +90.
default_elevation_deg = 0
default_azimuth_deg = 0
GPU_IDX = 0
init_token='werewolf'
token_name = f'_{init_token}_'
stable_version = '2.1'  #2.1 #2.0 #1.5
MODEL_NAME = "stabilityai/stable-diffusion-2-1-base" # "stabilityai/stable-diffusion-2-1-base" # "runwayml/stable-diffusion-v1-5"
DATA_DIR =image_path
OUTPUT_DIR = save_path
placeholder_token=token_name
polar_angle=1

In [ ]:
%run preprocess_image.py --path {image_path_without_rgba}

In [ ]:
import os
DATA_DIR, IMAGE_NAME = os.path.split(image_path)
FILENAME=IMAGE_NAME.split('.')[0]
!cp  '{DATA_DIR}/rgba.png' '{DATA_DIR}/{workspace}_rgba.png'
!cp  '{DATA_DIR}/depth.png' '{DATA_DIR}/{workspace}_depth.png'

In [ ]:
import os


cmd = f"""
textual-inversion/textual_inversion.py \
  --pretrained_model_name_or_path={MODEL_NAME} \
  --train_data_dir={DATA_DIR} \
  --learnable_property="object" \
  --placeholder_token={init_token} \
  --initializer_token={init_token} \
  --resolution=512 \
  --train_batch_size=16 \
  --gradient_accumulation_steps=1 \
  --max_train_steps=3000 \
  --lr_scheduler="constant" \
  --lr_warmup_steps=0 \
  --output_dir={OUTPUT_DIR} \
  --use_augmentations
  
"""
%run {cmd}


In [ ]:
guidance = f"""
guidance/sd_utils.py \
  --sd_version {stable_version} \
  --text="A high-resolution DSLR image of <token>" \
  --learned_embeds_path={OUTPUT_DIR} \
  --workspace={workspace}
"""
%run {guidance}


In [ ]:
import os

RUN_ID = "dmt"
RUN_ID2 = "nerf"
DATA_DIR, IMAGE_NAME = os.path.split(image_path)
FILENAME=IMAGE_NAME.split('.')[0]
step1 = True # change this according to your needs
step2 = True # change this according to your needs

In [ ]:

cmd= f"""
main.py -O \
        --text "A high-resolution DSLR image of <token>" \
        --sd_version {stable_version} \
        --image {image_path} \
        --learned_embeds_path {OUTPUT_DIR}/learned_embeds.bin \
        --workspace out/magic123-{RUN_ID}-coarse/$dataset/magic123_${FILENAME}_${RUN_ID}_coarse \
        --optim adam \
        --iters 5000 \
        --guidance SD zero123 \
        --lambda_guidance 1.0 40 \
        --guidance_scale 100 5 \
        --latent_iter_ratio 0 \
        --normal_iter_ratio 0.2 \
        --t_range 0.2 0.6 \
        --bg_radius -1 \
        --default_polar {polar_angle} \
        --save_mesh 
"""
if step1:
    %run {cmd}


In [ ]:

if step2:
    cmd2 = f"""
    main.py -O \
        --text "A high-resolution DSLR image of <token>" \
        --sd_version {stable_version} \
        --image {image_path} \
        --learned_embeds_path {OUTPUT_DIR}/learned_embeds.bin  \
        --workspace out/magic123-{RUN_ID}-{RUN_ID2}/$dataset/magic123_${FILENAME}_${RUN_ID}_${RUN_ID2} \
        --dmtet --init_ckpt out/magic123-{RUN_ID}-coarse/$dataset/magic123_${FILENAME}_${RUN_ID}_coarse/checkpoints/magic123_${FILENAME}_${RUN_ID}_coarse.pth \
        --iters 5000 \
        --optim adam \
        --latent_iter_ratio 0 \
        --guidance SD zero123 \
        --lambda_guidance 1e-3 0.01 \
        --guidance_scale 100 5 \
        --rm_edge \
        --bg_radius -1 \
        --default_polar {polar_angle} \
        --save_mesh
    """
    %run {cmd2}

In [ ]:
RUN_ID2="coarse"
path = f"""/notebooks/Magic123/out/magic123-{RUN_ID}-{RUN_ID2}/$dataset/magic123_${FILENAME}_${RUN_ID}_${RUN_ID2}"""
%cd {path}
!zip {workspace}.zip -r mesh
%cd ../

